In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform

In [2]:
df_commute = pd.read_csv('data/commute_data.csv')

In [3]:
df_tracts = pd.read_csv('data/census_tracts_2010.csv')

In [4]:
df_commute = df_commute.head(1000)

In [5]:
df_pivot = df_commute.pivot(index='OFIPS', columns='DFIPS', values='FLOW')

In [6]:
df_pivot = df_pivot.fillna(0)

In [ ]:
Z = linkage(distxy, linkage_method)
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=12.,  # font size for the x axis labels
    labels = labels,
    color_threshold = 20
)
plt.show()